In [1]:
import pandas as pd
import numpy as np
import os
import kaggle
import ast
from sklearn.cluster import KMeans
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import pickle
from scipy import spatial
import operator
from ast import literal_eval
from fuzzywuzzy import fuzz

d:\Anaconda\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#from kaggle.api.kaggle_api_extended import KaggleApi
#api = KaggleApi()
#api.authenticate()

#api.dataset_download_files('shuyangli94/food-com-recipes-and-user-interactions')

In [2]:
os.chdir(r"D:\recipe_recommender\recipe_recommender_webiste\food-com-recipes-and-user-interactions")
interactions_train = pd.read_csv("interactions_train.csv")
pp_recipes = pd.read_csv("PP_recipes.csv")
pp_users = pd.read_csv("PP_users.csv")
raw_interactions = pd.read_csv("RAW_interactions.csv")
raw_recipes = pd.read_csv("RAW_recipes.csv")

In [61]:
id_dict = {}

for i in range(len(pp_recipes)):
    id_dict[pp_recipes['i'][i]] = pp_recipes['id'][i]


In [97]:
pp_recipes[:10]

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"
5,465171,111231,"[40480, 3390, 829, 35873, 7047, 13731, 2640, 1...","[[13731, 30684, 260, 245, 17843, 25592, 10601]...","[40480, 40482, 7087, 13731, 30684, 260, 245, 5...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,"[6861, 7655, 6846, 6906, 1789, 131, 6863, 1833..."
6,163861,85356,"[40480, 1966, 488, 5218, 252, 5867, 10994, 118...","[[31801, 12395, 25808], [17918], [6953], [1133...","[40480, 40482, 604, 704, 15110, 244, 15684, 24...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,"[5574, 2683, 6270, 5319, 2499, 869, 1278, 4987..."
7,186383,105140,"[40480, 5317, 7, 491, 11274, 5639, 40481]","[[17918], [25916], [15473, 8361], [15473, 1016...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 332, 335, 6270, 800, 4987, 7470, ..."
8,116395,8671,"[40480, 16190, 13249, 4914, 5639, 40481]","[[17918], [36374, 3388, 650, 256, 6444], [2361...","[40480, 40482, 19093, 271, 40478, 40482, 23667...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",1,"[2683, 1689, 5687, 1098, 840, 7782, 7011, 1910..."
9,303460,160334,"[40480, 1287, 7912, 504, 22118, 19276, 831, 47...","[[559, 1164, 6020], [511, 532, 543, 241], [664...","[40480, 40482, 14259, 1055, 11, 4364, 488, 827...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[6413, 7997, 3148, 3710, 1799, 2007, 3203, 265..."


In [63]:
raw_recipes[raw_recipes['id'] == 137739]['ingredients']

0    ['winter squash', 'mexican seasoning', 'mixed ...
Name: ingredients, dtype: object

Turn each item into a row making item (the recipe) the target variable. The predictor variables will be n_steps, n_ingredients, ingredient_tokens, nutrition, minutes. A user will then fll out a form specifying the predictor variables and based on the predictor variables a model will predict what recipe they may like. 

In [64]:
pp_users.head()

,u,techniques,items,n_items,ratings,n_ratings
0,0,"[8, 0, 0, 5, 6, 0, 0, 1, 0, 9, 1, 0, 0, 0, 1, ...","[1118, 27680, 32541, 137353, 16428, 28815, 658...",31,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...",31
1,1,"[11, 0, 0, 2, 12, 0, 0, 0, 0, 14, 5, 0, 0, 0, ...","[122140, 77036, 156817, 76957, 68818, 155600, ...",39,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",39
2,2,"[13, 0, 0, 7, 5, 0, 1, 2, 1, 11, 0, 1, 0, 0, 1...","[168054, 87218, 35731, 1, 20475, 9039, 124834,...",27,"[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ...",27
3,3,"[498, 13, 4, 218, 376, 3, 2, 33, 16, 591, 10, ...","[163193, 156352, 102888, 19914, 169438, 55772,...",1513,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ...",1513
4,4,"[161, 1, 1, 86, 93, 0, 0, 11, 2, 141, 0, 16, 0...","[72857, 38652, 160427, 55772, 119999, 141777, ...",376,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ...",376


In [65]:
new_df = pd.DataFrame()
recipes, ratings = [], []

for i in range(len(pp_users)):
    item = ast.literal_eval(pp_users['items'][i])
    rating = ast.literal_eval(pp_users['ratings'][i])

    for j in range(len(item)):
        recipes.append(id_dict[item[j]])
        ratings.append(rating[j])

new_df['recipe'] = recipes
new_df['rating'] = ratings

In [66]:
most_rated = list(dict(new_df['recipe'].value_counts()[0:50000]).keys())

In [67]:
new_df2 = new_df[new_df['recipe'].isin(most_rated)]
new_df2 = new_df2.reset_index(drop=True)
new_df2 = new_df2.groupby(['recipe']).mean()
new_df2 = new_df2.reset_index(drop=False)
new_df2 = new_df2[new_df2['rating'] > 3]

raw_recipes2 = raw_recipes[raw_recipes['id'].isin(list(new_df2['recipe']))]
raw_recipes2 = raw_recipes2.reset_index(drop=True)

new_df2 = new_df2.reset_index(drop=True)

In [68]:
nutrition, minutes, ingreds, tags = {},{},{},{}

for i in range(len(raw_recipes2)):
    recipe = raw_recipes2['id'][i]
    key_words = ast.literal_eval(raw_recipes2['tags'][i])
    ingredients = ast.literal_eval(raw_recipes2['ingredients'][i])
    nutr = raw_recipes2['nutrition'][i]
    mins = raw_recipes2['minutes'][i]

    
    nutrition[recipe] = nutr
    minutes[recipe] = mins
    ingreds[recipe] = ingredients
    tags[recipe] = key_words

nutrition2, minutes2, ingreds2, tags2 = [],[],[],[]

for j in range(len(new_df2)):
    recipe = new_df2['recipe'][j]

    tags2.append(tags[recipe])
    nutrition2.append(nutrition[recipe])
    minutes2.append(minutes[recipe])
    ingreds2.append(ingreds[recipe])


new_df2['tags'] = tags2
new_df2['nutrition'] = nutrition2
new_df2['minutes'] = minutes2
new_df2['ingredients'] = ingreds2


In [69]:
new_df2.head()

,recipe,rating,tags,nutrition,minutes,ingredients
0,40,4.800000,"[60-minutes-or-less, time-to-make, course, pre...","[311.1, 0.0, 308.0, 0.0, 0.0, 0.0, 27.0]",35,"[sugar, lemons, rind of, fresh water, fresh le..."
1,49,4.000000,"[weeknight, time-to-make, course, main-ingredi...","[627.7, 38.0, 8.0, 35.0, 115.0, 64.0, 4.0]",75,"[fresh mushrooms, butter, boneless skinless ch..."
2,58,4.333333,"[weeknight, time-to-make, course, main-ingredi...","[280.1, 9.0, 36.0, 24.0, 59.0, 7.0, 7.0]",164,"[beef eye round, vegetable oil, dried thyme le..."
3,62,4.666667,"[30-minutes-or-less, time-to-make, course, mai...","[407.8, 23.0, 17.0, 0.0, 34.0, 11.0, 18.0]",25,"[fresh lemon juice, olive oil, black beans, fr..."
4,66,4.933333,"[lactose, 30-minutes-or-less, time-to-make, co...","[772.0, 6.0, 657.0, 93.0, 13.0, 2.0, 63.0]",30,"[brewed coffee, ketchup, red wine vinegar, dar..."


In [70]:
# All recipes in df are rated > 3

new_df2 = new_df2.drop('rating',axis=1)  

In [71]:
final_df = new_df2.copy()
final_df.head() 

,recipe,tags,nutrition,minutes,ingredients
0,40,"[60-minutes-or-less, time-to-make, course, pre...","[311.1, 0.0, 308.0, 0.0, 0.0, 0.0, 27.0]",35,"[sugar, lemons, rind of, fresh water, fresh le..."
1,49,"[weeknight, time-to-make, course, main-ingredi...","[627.7, 38.0, 8.0, 35.0, 115.0, 64.0, 4.0]",75,"[fresh mushrooms, butter, boneless skinless ch..."
2,58,"[weeknight, time-to-make, course, main-ingredi...","[280.1, 9.0, 36.0, 24.0, 59.0, 7.0, 7.0]",164,"[beef eye round, vegetable oil, dried thyme le..."
3,62,"[30-minutes-or-less, time-to-make, course, mai...","[407.8, 23.0, 17.0, 0.0, 34.0, 11.0, 18.0]",25,"[fresh lemon juice, olive oil, black beans, fr..."
4,66,"[lactose, 30-minutes-or-less, time-to-make, co...","[772.0, 6.0, 657.0, 93.0, 13.0, 2.0, 63.0]",30,"[brewed coffee, ketchup, red wine vinegar, dar..."


In [72]:
ings = []

for i in range(len(final_df)):
    #ing_list = ast.literal_eval(new_df2['ingredients'][i])
    ings.extend(final_df['ingredients'][i])

cnt = Counter(ings)
ingredient_list = sorted(cnt, key=cnt.get, reverse=True)[0:1100]

#with open(r'D:\recipe_recommender\recipe_recommender_webiste\recommender_app\static\ingredients2.pickle', 'wb') as f: 
#    pickle.dump(ingredient_list, f)

In [73]:
ethnicity = ['spanish','egyptian','tex-mex','welsh','australian','polynesian','pacific-northwest',
'swiss','italian','german','sudanese','chilean','costa-rican','hawaiian','malaysian','icelandic','norwegian',
'pennsylvania-dutch','somalian','pakistani','nepalese','venezuelan','asian','puerto-rican','nigerian',
'ethiopian','vietnamese','hungarian','laotian','dutch','chinese','south-african','brazilian','austrian',
'greek','middle-eastern','iraqi','ecuadorean','creole','african','korean','cambodian','caribbean','thai',
'indonesian','south-american','midwestern','north-american','mongolian','libyan','new-zealand','russian',
'congolese','mexican','south-west-pacific','japanese','native-american','portuguese','palestinian','cuban',
'scottish','central-american','filipino','indian','honduran','european','saudi-arabian','guatemalan','argentine',
'irish','turkish','georgian','british-columbian','peruvian','scandinavian','southern-united-states','lebanese',
'czech','canadian','cajun','northeastern-united-states','colombian','southwestern-united-states']

delete = ['cocktails','smoothies','beverages'] 

dietary_restrict = ['vegan','vegetarian','dairy-free','gluten-free','nut-free']

types_of_food = ['spreads','omelets-and-frittatas','frozen-desserts','dips','salsas',
'shakes','appetizers','sandwiches','cobblers-and-crisps','soul',
'seafood','pies','salads','barbecue','stews','side-dishes']

holiday = ['super-bowl','rosh-hashanah','new-years','mothers-day','st-patricks-day',
'fathers-day','valentines-day','ramadan','chinese-new-year','easter','christmas',
'april-fools-day','cinco-de-mayo','independence-day','labor-day','memorial-day',
'passover','kwanzaa','thanksgiving','hanukkah','halloween']


In [74]:
delete_recipes,ethnicity_lst,type_of_food_lst,holiday_lst,dr_lst = [],[],[],[],[]

for i in range(len(final_df)):
    tags = final_df['tags'][i]

    ethnicity_check = False
    type_of_food_check = False
    holiday_check = False
    dr_check = False

    ethnicity_lst2,type_of_food_lst2,holiday_lst2,dr_lst2 = [],[],[],[]

    for j in range(len(tags)):

        if tags[j] in delete:
            delete_recipes.append(final_df['recipe'][i])
            
        if tags[j] in ethnicity:
            ethnicity_lst2.append(tags[j])
            ethnicity_check = True

        if tags[j] in types_of_food:
            type_of_food_lst2.append(tags[j])
            type_of_food_check = True

        if tags[j] in holiday:
            holiday_lst2.append(tags[j])
            holiday_check = True

        if tags[j] in dietary_restrict:
            dr_lst2.append(tags[j])
            dr_lst_check = True


    if ethnicity_check == False:
        ethnicity_lst2.append("N/A")
    
    if type_of_food_check == False:
        type_of_food_lst2.append("N/A")

    if holiday_check == False:
        holiday_lst2.append("N/A")

    if dr_check == False:
        dr_lst2.append("N/A")

    ethnicity_lst.append(ethnicity_lst2)
    type_of_food_lst.append(type_of_food_lst2)
    holiday_lst.append(holiday_lst2)
    dr_lst.append(dr_lst2)


final_df['ethnicity'] = ethnicity_lst
final_df['type_of_food'] = type_of_food_lst
final_df['holiday'] = holiday_lst
final_df['dietary_restriction'] = dr_lst

final_df = final_df[~final_df['recipe'].isin(delete_recipes)]
final_df = final_df.reset_index(drop=True)

In [75]:
ingredients_bin = []

for i in range(len(final_df['ingredients'])):
    binaryList = []
    for ingred in ingredient_list:
        if ingred in final_df['ingredients'][i]:
            binaryList.append(1)
        else:
            binaryList.append(0)
            
    ingredients_bin.append(binaryList)
    
    

In [76]:
final_df['ingredients_bin'] = ingredients_bin
final_df = final_df.set_index("recipe")
final_df['nutrition'] = final_df.nutrition.apply(lambda x: literal_eval(x))
final_df = final_df[final_df['minutes'] < 3000]


In [77]:
cals,sug,prot,car = [],[],[],[]
nutrition_lst = list(final_df['nutrition'])

for i in range(len(nutrition_lst)):
    cals.append(nutrition_lst[i][0])
    sug.append(nutrition_lst[i][2])
    prot.append(nutrition_lst[i][4])
    car.append(nutrition_lst[i][6])

final_df['calories'] = cals
final_df['sugar'] = sug
final_df['protein'] = prot
final_df['carbs'] = car

In [78]:
recipe_names = {}
recipe_steps = {}
recipe_description = {}
for i in range(len(raw_recipes2)):
    recipe_names[raw_recipes2['id'][i]] = raw_recipes2['name'][i]
    recipe_steps[raw_recipes2['id'][i]] = raw_recipes2['steps'][i]
    recipe_description[raw_recipes2['id'][i]] = raw_recipes2['description'][i]

In [80]:
final_df = final_df.reset_index()
recipe_name,recipe_step,recipe_desc = [],[],[]
for i in range(len(final_df)):
    recipe_name.append(recipe_names[final_df['recipe'][i]])

for j in range(len(final_df)):
    recipe_step.append(recipe_steps[final_df['recipe'][j]])

for k in range(len(final_df)):
    recipe_desc.append(recipe_description[final_df['recipe'][k]])


final_df['recipe_name'] = recipe_name
final_df['recipe_steps'] = recipe_step
final_df['description'] = recipe_desc

final_df = final_df.set_index('recipe')


In [81]:
final_df.head()


,tags,nutrition,minutes,ingredients,ethnicity,type_of_food,holiday,dietary_restriction,ingredients_bin,calories,sugar,protein,carbs,recipe_name,recipe_steps,description
recipe,,,,,,,,,,,,,,,,
49,"[weeknight, time-to-make, course, main-ingredi...","[627.7, 38.0, 8.0, 35.0, 115.0, 64.0, 4.0]",75,"[fresh mushrooms, butter, boneless skinless ch...","[european, italian]",[N/A],[N/A],[N/A],"[1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",627.7,8.0,115.0,4.0,chicken breasts lombardi,['cook mushrooms in 2 tbsp butter in a large s...,"cheese, chicken, and mushrooms with marsala."
58,"[weeknight, time-to-make, course, main-ingredi...","[280.1, 9.0, 36.0, 24.0, 59.0, 7.0, 7.0]",164,"[beef eye round, vegetable oil, dried thyme le...",[N/A],[stews],[N/A],"[gluten-free, N/A]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...",280.1,36.0,59.0,7.0,low fat burgundy beef vegetable stew,"['trim fat from beef , cut into 1-inch pieces'...",(high fiber)
62,"[30-minutes-or-less, time-to-make, course, mai...","[407.8, 23.0, 17.0, 0.0, 34.0, 11.0, 18.0]",25,"[fresh lemon juice, olive oil, black beans, fr...",[north-american],[salads],[N/A],"[vegetarian, gluten-free, N/A]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",407.8,17.0,34.0,18.0,black bean corn and tomato salad,"['in a bowl whisk together lemon juice , oil ,...","this is easy, delicious, colorful, delicious, ..."
66,"[lactose, 30-minutes-or-less, time-to-make, co...","[772.0, 6.0, 657.0, 93.0, 13.0, 2.0, 63.0]",30,"[brewed coffee, ketchup, red wine vinegar, dar...","[north-american, southern-united-states]",[N/A],[N/A],[N/A],"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",772.0,657.0,13.0,63.0,black coffee barbecue sauce,['combine all ingredients in a saucepan and si...,it's great to know folks like this sauce so mu...
91,"[weeknight, time-to-make, course, main-ingredi...","[412.6, 37.0, 10.0, 17.0, 16.0, 81.0, 14.0]",150,"[brown rice, chicken broth, unsalted butter, o...",[N/A],[side-dishes],[N/A],[N/A],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",412.6,10.0,16.0,14.0,brown rice and vegetable pilaf,"['preheat oven to 325 degrees f', 'heat chicke...",this is good with almost anything... robb


In [ ]:
def Similarity(user_input, row1):

    ingreds_A = row1['ingredients_bin']
    ingreds_B = user_input['ingredients_bin'][user_input.index[0]]

    IngredDistance = spatial.distance.cosine(ingreds_A, ingreds_B)
    
    nutrition_A = row1['nutrition']
    nutrition_B = user_input['nutrition'][user_input.index[0]]

    NutDistance = spatial.distance.cosine(nutrition_A, nutrition_B)

    return IngredDistance + NutDistance 

In [ ]:
def filter_df(df,time_filter, dietary_restriction, ethnicity, type_of_food, holiday):

    if time_filter == "Less than 5 Hours":
        df = df[df['minutes'] < 300]
    if time_filter == "Less than 3 Hours":
        df = df[df['minutes'] < 180]
    if time_filter == "Less than 1 Hour":
        df = df[df['minutes'] < 60]
    if time_filter == "Any Time":
        df = df

    if dietary_restriction != "None":
        df = df.explode('dietary_restriction')
        df = df[df['dietary_restriction'].isin(dietary_restriction)]

    if ethnicity != "Any":
        df = df.explode('ethnicity')
        df = df[df['ethnicity'] == ethnicity]

    if type_of_food != "Any":
        df = df.explode('type_of_food')
        df = df[df['type_of_food'] == type_of_food]

    if holiday != "No":
        df = df.explode('holiday')
        df = df[df['holiday'] == holiday]

    return df

In [ ]:
def getNeighbors(user_input, df, K, name_df):
    distances = []
    
    for index, recipe in df.iterrows():
        
        dist = Similarity(user_input, recipe)
        name = name_df[name_df['id'] == index]['name']
        steps = name_df[name_df['id'] == index]['steps']
        distances.append((name,steps, dist))

    distances.sort(key=operator.itemgetter(2))
    neighbors = []

    for x in range(K):
        neighbors.append(distances[x])
    return neighbors

In [25]:
ingredients2,recipe_steps2,description2,recipe_name2 = [],[],[],[]
final_df = final_df.reset_index()

for i in range(len(final_df)):
    ings = [j.title() for j in final_df['ingredients'][i]]
    ings2 = ", ".join(ings)
    ingredients2.append(ings)

    steps = ast.literal_eval(final_df['recipe_steps'][i])
    steps2 = [j.title() for j in steps]
    steps3 = ", ".join(steps2)
    recipe_steps2.append(steps3)

    desc = str(final_df['description'][i]).title()
    description2.append(desc)

    recipe_name = str(final_df['recipe_name'][i]).title()
    recipe_name2.append(recipe_name)


final_df['ingredients_html'] = ingredients2
final_df['steps_html'] = recipe_steps2
final_df['description_html'] = description2
final_df['recipe_name_html'] = recipe_name2

In [27]:
tgs = []

for i in range(len(final_df)):
    #ing_list = ast.literal_eval(new_df2['ingredients'][i])
    tgs.extend(final_df['tags'][i])

cnt = Counter(tgs)
tag_list = sorted(cnt, key=cnt.get, reverse=True)

In [28]:
tags_bin = []

for i in range(len(final_df['tags'])):
    binaryList = []
    for tag in tag_list:
        if tag in final_df['tags'][i]:
            binaryList.append(1)
        else:
            binaryList.append(0)
            
    tags_bin.append(binaryList)

In [29]:
final_df['tag_bins'] = tags_bin

In [31]:
final_df = final_df.set_index("recipe")

In [34]:
final_df = final_df.drop("index",axis=1)

In [35]:
final_df.head()

,tags,nutrition,minutes,ingredients,ethnicity,type_of_food,holiday,dietary_restriction,ingredients_bin,calories,...,protein,carbs,recipe_name,recipe_steps,description,ingredients_html,steps_html,description_html,recipe_name_html,tag_bins
recipe,,,,,,,,,,,,,,,,,,,,,
49,"[weeknight, time-to-make, course, main-ingredi...","[627.7, 38.0, 8.0, 35.0, 115.0, 64.0, 4.0]",75,"[fresh mushrooms, butter, boneless skinless ch...","[european, italian]",[N/A],[N/A],[N/A],"[1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",627.7,...,115.0,4.0,chicken breasts lombardi,['cook mushrooms in 2 tbsp butter in a large s...,"cheese, chicken, and mushrooms with marsala.","[Fresh Mushrooms, Butter, Boneless Skinless Ch...",Cook Mushrooms In 2 Tbsp Butter In A Large Ski...,"Cheese, Chicken, And Mushrooms With Marsala.",Chicken Breasts Lombardi,"[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, ..."
58,"[weeknight, time-to-make, course, main-ingredi...","[280.1, 9.0, 36.0, 24.0, 59.0, 7.0, 7.0]",164,"[beef eye round, vegetable oil, dried thyme le...",[N/A],[stews],[N/A],"[gluten-free, N/A]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...",280.1,...,59.0,7.0,low fat burgundy beef vegetable stew,"['trim fat from beef , cut into 1-inch pieces'...",(high fiber),"[Beef Eye Round, Vegetable Oil, Dried Thyme Le...","Trim Fat From Beef , Cut Into 1-Inch Pieces, I...",(High Fiber),Low Fat Burgundy Beef Vegetable Stew,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, ..."
62,"[30-minutes-or-less, time-to-make, course, mai...","[407.8, 23.0, 17.0, 0.0, 34.0, 11.0, 18.0]",25,"[fresh lemon juice, olive oil, black beans, fr...",[north-american],[salads],[N/A],"[vegetarian, gluten-free, N/A]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",407.8,...,34.0,18.0,black bean corn and tomato salad,"['in a bowl whisk together lemon juice , oil ,...","this is easy, delicious, colorful, delicious, ...","[Fresh Lemon Juice, Olive Oil, Black Beans, Fr...","In A Bowl Whisk Together Lemon Juice , Oil , A...","This Is Easy, Delicious, Colorful, Delicious, ...",Black Bean Corn And Tomato Salad,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, ..."
66,"[lactose, 30-minutes-or-less, time-to-make, co...","[772.0, 6.0, 657.0, 93.0, 13.0, 2.0, 63.0]",30,"[brewed coffee, ketchup, red wine vinegar, dar...","[north-american, southern-united-states]",[N/A],[N/A],[N/A],"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",772.0,...,13.0,63.0,black coffee barbecue sauce,['combine all ingredients in a saucepan and si...,it's great to know folks like this sauce so mu...,"[Brewed Coffee, Ketchup, Red Wine Vinegar, Dar...",Combine All Ingredients In A Saucepan And Simm...,It'S Great To Know Folks Like This Sauce So Mu...,Black Coffee Barbecue Sauce,"[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, ..."
91,"[weeknight, time-to-make, course, main-ingredi...","[412.6, 37.0, 10.0, 17.0, 16.0, 81.0, 14.0]",150,"[brown rice, chicken broth, unsalted butter, o...",[N/A],[side-dishes],[N/A],[N/A],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",412.6,...,16.0,14.0,brown rice and vegetable pilaf,"['preheat oven to 325 degrees f', 'heat chicke...",this is good with almost anything... robb,"[Brown Rice, Chicken Broth, Unsalted Butter, O...","Preheat Oven To 325 Degrees F, Heat Chicken Br...",This Is Good With Almost Anything... Robb,Brown Rice And Vegetable Pilaf,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [36]:
final_df.to_pickle(r"D:\recipe_recommender\recipe_recommender_webiste\recommender_app\static\recipe_data.pkl")